In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
import sys
sys.path.append('modules')

import importlib
import data

In [49]:
importlib.reload(sys.modules['data'])

<module 'data' from '/Users/vladparakhin/Documents/GitHub/ML_Gradient_Boosting_Classifier/modules/data.py'>

In [50]:
from data import set_filepath, test_loc, percent_missing, display_test

In [42]:
temp = pd.read_csv(transform_final)
temp.loc[231]

VehListdays                             48.50059
VehMileage                                  64.0
Dealer_Listing_Price                     38900.0
VehYear                                     2019
ListingID                                 311804
VehEngine                  3.6L V6 24V MPFI DOHC
SellerListSrc           Digital Motorworks (DMi)
SellerState                                   PA
VehFuel                                 Gasoline
VehColorExt                                White
VehColorInt                                black
VehTransmission                8-Speed Automatic
VehPriceLabel                          Good Deal
VehDriveTrain                                4WD
Vehicle_Trim                             Limited
VehMake                                     Jeep
VehModel                          Grand Cherokee
RatingCategory                  Excellent Rating
SellerCategory                      Rare Sellers
SellerIsPriv                               False
SourceCategory      

## Train

In [44]:
stage = 'train'

In [ ]:
file_path_lists = set_filepath(stage)
transform_final = result[4]

In [5]:
price_train_final_df = pd.read_csv(transform_final)

In [7]:
price_train_final_df.columns

Index(['VehListdays', 'VehMileage', 'Dealer_Listing_Price', 'VehYear',
       'ListingID', 'VehEngine', 'SellerListSrc', 'SellerState', 'VehFuel',
       'VehColorExt', 'VehColorInt', 'VehTransmission', 'VehPriceLabel',
       'VehDriveTrain', 'Vehicle_Trim', 'VehMake', 'VehModel',
       'RatingCategory', 'SellerCategory', 'SellerIsPriv', 'SourceCategory',
       'VehCertified'],
      dtype='object')

In [17]:
percent_nan = percent_missing(price_train_final_df)
percent_nan

Series([], dtype: float64)

In [18]:
price_train_final_df.drop(columns=['ListingID'], inplace=True)

In [20]:
X_clean = price_train_final_df.drop('Dealer_Listing_Price',axis=1)
y_clean = price_train_final_df['Dealer_Listing_Price']

In [21]:
X_clean_encoded = pd.get_dummies(X_clean, drop_first=True)

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_clean_encoded, y_clean, test_size=0.10, random_state=101)

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()

In [26]:
scaled_X_train_clean = scaler.fit_transform(X_train_clean)
scaled_X_test_clean = scaler.transform(X_test_clean)

In [27]:
scaled_X_train_clean.shape, scaled_X_test_clean.shape

((5668, 121), (630, 121))

In [28]:
from sklearn.linear_model import ElasticNet

In [29]:
base_elastic_model = ElasticNet()

In [30]:
param_grid = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
grid_model = GridSearchCV(estimator=base_elastic_model,
                          param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          verbose=1)

In [33]:
grid_model.fit(scaled_X_train_clean,y_train_clean)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.696e+09, tolerance: 2.646e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.680e+09, tolerance: 2.471e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [34]:
grid_model.best_params_

{'alpha': 10, 'l1_ratio': 1}

In [35]:
y_pred = grid_model.predict(scaled_X_test_clean)

In [36]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [37]:
mean_absolute_error(y_test_clean,y_pred)

1949.4371102388482

In [38]:
np.sqrt(mean_squared_error(y_test_clean,y_pred))

2603.075458011679

In [40]:
np.mean(price_train_final_df['Dealer_Listing_Price'])

32265.05287392823

 ## Test

In [67]:
stage = 'test'

In [68]:
file_path_lists = set_filepath(stage)
transform_final = file_path_lists[4]

In [69]:
test_final_df = pd.read_csv(transform_final)

In [70]:
test_final_df.columns

Index(['VehListdays', 'VehMileage', 'VehYear', 'ListingID', 'VehEngine',
       'SellerListSrc', 'SellerState', 'VehFuel', 'VehColorExt', 'VehColorInt',
       'VehTransmission', 'VehPriceLabel', 'VehDriveTrain', 'VehMake',
       'VehModel', 'RatingCategory', 'SellerCategory', 'SellerIsPriv',
       'SourceCategory', 'VehCertified'],
      dtype='object')

In [71]:
X_test_final = pd.get_dummies(test_final_df, drop_first=True)

In [72]:
# Cross check and recon the columns
missing_cols = set(X_train_clean.columns) - set(X_test_final.columns)
for col in missing_cols:
    X_test_final[col] = 0

In [73]:
X_test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 123 entries, VehListdays to VehFuel_unknown
dtypes: bool(2), float64(2), int64(22), uint8(97)
memory usage: 284.3 KB


In [74]:
X_test_final = X_test_final[X_train_clean.columns]

In [75]:
scaled_X_test_final = scaler.transform(X_test_final)

In [76]:
predictions = grid_model.predict(scaled_X_test_final)

In [77]:
report_df = pd.read_csv('data/report/report.csv')

In [78]:
report_df['Dealer_Listing_Price'] = predictions

In [79]:
report_df.to_csv('report.csv', index=False)

In [80]:
report_df.head()

,ListingID,Vehicle_Trim,Dealer_Listing_Price
0,8622015,FWD,34368.255796
1,8625693,Limited,26989.157438
2,8625750,Limited,19956.414154
3,8626885,Limited,23242.270791
4,8627430,Luxury,35480.710098
